<a href="https://colab.research.google.com/github/nfrn/Deep-Learning-for-Health-Text-Mining/blob/master/Tutorial_1_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Interesting WebSites for Deep Learnig**

*   http://scs.ryerson.ca/~aharley/vis/conv/flat.html
*   https://playground.tensorflow.org




In [0]:
!pip install keras_tqdm
!pip install matplotlib==3.1.0
!pip install shap
!pip install lime
!pip install eli5

### **Breast Cancer Wisconsin (Diagnostic) Data Set**
Processing data from:

https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/

We read in the data and do some basic cleanup for missing values. For the description of the fields, see :

https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names

In summary:
```
 Sample code number          : Id number (not used and thus dropped)
 Clump Thickness             : 1–10
 Uniformity of Cell Size     : 1–10
 Uniformity of Cell Shape    : 1–10
 Marginal Adhesion           : 1–10
 Single Epithelial Cell Size : 1–10
 Bare Nuclei                 : 1–10
 Bland Chromatin             : 1–10
 Normal Nucleoli             : 1–10
 Mitoses                     : 1–10
 Class                       : 2 for benign, 4 for malignant
```



In [0]:
import pandas as pd
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data")
data = data.drop(data.columns[0], axis=1)
data.head(10)

###**Data Cleaning and Preprocessing**

In [0]:
df = pd.DataFrame(data)
# Notice that in column 6 there are some missing values.
print(df.loc[df[6] == "?"].head(5))

# We calculate the mean of that feature.
df_6_without_missing_values = df[6].loc[df[6] != "?"]
mean = df_6_without_missing_values.astype(int).mean()
print("Mean value: " + str(mean))

# Replace missing values with mean value
df[6] = df[6].replace("?", mean)
df[6] = df[6].astype(int)

# Re-arranging labels 2 -> 0 and 4 -> 1
df[10] = df[10].replace(2,0).replace(4,1)

###**Exploratory Data Analysis (EDA)**

In [0]:
names = [ "ID", "Clump thickness", "Uniformity of Cell Size", "Uniformity of Cell Shape", "Marginal Adhesion", "Single Epithelial Cell Size", "Bare Nuclei", "Bland Chromatin", "Normal Nucleoli", "Mitoses", "Class" ]
df.columns = names[1:]
hists = df.hist(bins=20, figsize=(15,20))
import matplotlib.pyplot as plt
plt.matshow(df.corr())
plt.show()

In [0]:
# Seaborn visualization library
import seaborn as sns
# Create the default pairplot
sns.pairplot(df, hue = 'Class', diag_kind = 'kde',
             plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
             height = 4)


# The diagonal diagrams describe
# The other diagrams describe the relationship (or lack thereof) between two variable

In [0]:
# Using information from all columns (0-9) to predict target (column 10)
X = df.iloc[:, :9]
Y = df.iloc[:, 9]

# Splitting between traning and testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

# Checking the shapes to get an understanding of the problem
print( X_train.shape, X_test.shape )
print( Y_train.shape, Y_test.shape )

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier()
dtree.fit(X_train,Y_train)

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

Y_pred = dtree.predict(X_test)
Y_pred = [ 1 if y>=0.5 else 0 for y in Y_pred]
cm = confusion_matrix(Y_test, Y_pred)

df_cm = pd.DataFrame(cm, index = ["Benign", "Malign"],
                  columns = ["Benign", "Malign"])
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16})
plt.show()

print(accuracy_score(Y_test, Y_pred))

In [0]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [0]:
import numpy as np
import matplotlib.pyplot as plt
# Feature importance dataframe
imp_df = pd.DataFrame({'feature': ["Clump Thickness","Uniformity of Cell Size ","Uniformity of Cell Shape", "Marginal Adhesion", "Single Epithelial Cell Size", "Bare Nucleoli", "Bland Chromatin"," Normal Nucleoli", "Mitoses"],
                       'importance': dtree.feature_importances_})
 
# Reorder by importance
ordered_df = imp_df.sort_values(by='importance')
imp_range=range(1,len(imp_df.index)+1)
 
## Barplot with confidence intervals
height = ordered_df['importance']
bars = ordered_df['feature']
y_pos = np.arange(len(bars))


# Create horizontal bars
plt.barh(y_pos, height)
 
# Create names on the y-axis
plt.yticks(y_pos, bars)

plt.xlabel("Reduction in tree impurity in decision tree")

plt.tight_layout()
# Show graphic
plt.show()



In [0]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.metrics.scorer import make_scorer

import matplotlib.pyplot as plt
# Feature importance based on TRAINING set

perm_test = PermutationImportance(dtree, scoring=make_scorer(accuracy_score),
                                   n_iter=50, cv="prefit")

# fit and see the permuation importances
perm_test.fit(X_train, Y_train)

imp_df = eli5.explain_weights_df(perm_test)
label_df = pd.DataFrame({'feature': [ "x" + str(i) for i in range(9)], 'feature_name': ["Clump Thickness","Uniformity of Cell Size ","Uniformity of Cell Shape", "Marginal Adhesion", "Single Epithelial Cell Size", "Bare Nucleoli", "Bland Chromatin"," Normal Nucleoli", "Mitoses"]})
imp_df = pd.merge(label_df, imp_df, on='feature', how='inner', validate="one_to_one")
 
# Reorder by importance
ordered_df = imp_df.sort_values(by='weight')
imp_range=range(1,len(imp_df.index)+1)
 

## Barplot with confidence intervals

height = ordered_df['weight']
bars = ordered_df['feature_name']
ci = 1.96 * ordered_df['std']
y_pos = np.arange(len(bars))

# Create horizontal bars
plt.barh(y_pos, height, xerr=ci)
 
# Create names on the y-axis
plt.yticks(y_pos, bars)

plt.xlabel("Permutation feature importance training set (decrease in accuracy)")
plt.tight_layout()

# Show graphic
plt.show()

In [0]:
import lime
import lime.lime_tabular
import matplotlib.pyplot as plt

X_explain = X_test
explainer = lime.lime_tabular.LimeTabularExplainer(training_data=X_train,
                                                   feature_names=["Clump Thickness","Uniformity of Cell Size ","Uniformity of Cell Shape", "Marginal Adhesion", "Single Epithelial Cell Size", "Bare Nucleoli", "Bland Chromatin"," Normal Nucleoli", "Mitoses"],
                                                   discretize_continuous='auto', 
                                                   class_names=["benign", "malign"],
                                                   mode="classification",
                                                   verbose=True)

#Explaining first subject in test set using all 30 features
exp = explainer.explain_instance(X_explain[21,:], dtree.predict_proba, 
                                 num_features=10)
#Plot local explanation
plt = exp.as_pyplot_figure()
#plt.tight_layout()
exp.show_in_notebook(show_table=True)

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow.compat.v1 as tf


from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras_tqdm import TQDMNotebookCallback

model = Sequential()
model.add(Dense(16, input_dim=(9), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(X_train, Y_train,batch_size=16,validation_split=0.2, epochs=30,verbose=0,callbacks=[TQDMNotebookCallback(leave_inner=True)])


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

Y_pred = model.predict(X_test,verbose=0)
Y_pred = [ 1 if y>=0.5 else 0 for y in Y_pred]
cm = confusion_matrix(Y_test, Y_pred)

df_cm = pd.DataFrame(cm, index = ["Benign", "Malign"],
                  columns = ["Benign", "Malign"])
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16})
plt.show()
print(accuracy_score(Y_test, Y_pred))

In [0]:
import lime
import lime.lime_tabular
import matplotlib.pyplot as plt
import numpy as np

def predict(qc):
    global model
    prediction_Class_1 = model.predict_proba(qc) 
    x = np.zeros((prediction_Class_1.shape[0], 1))
    probability = (x + 1) - prediction_Class_1
    final = np.append(probability,prediction_Class_1, axis=1)
    return final

X_explain = X_test
explainer = lime.lime_tabular.LimeTabularExplainer(training_data=X_train,
                                                   feature_names=["Clump Thickness","Uniformity of Cell Size ","Uniformity of Cell Shape", "Marginal Adhesion", "Single Epithelial Cell Size", "Bare Nucleoli", "Bland Chromatin"," Normal Nucleoli", "Mitoses"],
                                                   discretize_continuous=True,
                                                   class_names=["benign", "malign"],
                                                   mode="classification",
                                                   verbose=True)
print(np.array([X_test[0,:]]).shape)
print(model.predict_proba(np.array([X_test[0,:]])))

#Explaining first subject in test set using all 30 features
exp = explainer.explain_instance(X_test[21,:], predict,num_features=10)
#Plot local explanation
plt = exp.as_pyplot_figure()
#plt.tight_layout()
exp.show_in_notebook(show_table=True)